# How to start

Before starting you must:
- Have conda installed
- `conda env create -f ess-notebooks-latest.yml python=3.7` . The yaml environment file is part of this repository.
- fetch the data `git clone git@github.com:scipp/ess-notebooks-data.git` somewhere local 
- Generate the `dataconfig.py` file using `make_config.py` located in same directory as this notebook. In general, you simply need to point `make_config.py` to the root directory of data you cloned above. Refer to the help `make_config.py --help` for more information. 

Converted to use scipp and notebook from [this repository](https://github.com/scipp/ess-legacy).

For Table of Contents install Jupyter extensions then reload the notebook:
`conda install -c conda-forge jupyter_contrib_nbextensions`
`jupyter contrib nbextension install --user`
`jupyter nbextension enable toc2/main`

# Experimental Summary

This script has been developed to measure local strain ε defined as ε = ΔL/L0 in a FCC steel sample under elastic strain in a stress rig. Measured at V20, HZB, Berlin, September 2018 by Peter Kadletz.

λ = 2dsinθ, where 2θ = π (transmission), edges characterise the Bragg condition and hence λ = 2d. Therefore strain is easily computed from the wavelength measurement of of a Bragg edge directly, using un-loaded vs loaded experimental runs (and reference mesurements). The known Miller indices of the crystal structure (FCC) are used to predict the wavelength where the Bragg edges should exist, which is bound by the reachable wavelength extents for the instrument. This provides an approximate region to apply a fit.  A complement error function is used to fit each Bragg edge, and a refined centre location (λ) for the edge is used in the strain measurement. Because each bragg edge can be identified individually, one can determine anisotropic strain across the unit cell in the reachable crystallographic directions. In addition the image processing allows for spacial grouping so localised effects, such as those on unconstrained edges of the sample or in necking regions of the sample can be treated seperately. The plotted outputs in the script aim to capture this.

# Setup

In [ ]:
import scipp as sc
import numpy as np
from scipp.plot import plot
import matplotlib.pyplot as plt
import utils
from utils import wfm

# Reduction

## Load the data files and instrument geometry

In [ ]:
ds = utils.load()

In [ ]:
ds

## Raw data visualization

In [ ]:
# p = plot(ds["sample"])
# p

In [ ]:
# p.make_static()

In [ ]:
# sc.neutron.instrument_view(ds["sample"])

## Converting time coordinate to TOF

Use the instrument geometry and chopper cascade parameters to compute time-distance diagram.

In [ ]:
plt.ion()
v20setup = wfm.v20.setup()
# Feed in the detector position to the instrument chopper cascade
v20setup['info']['detector_positions'] = {
    "GP2": -ds.coords['source-position'].value[2] + v20setup['info']['wfm_choppers_midpoint'] + sc.geometry.z(
    ds.coords["position"])['x', 0]['y', 0].value}

frames = wfm.get_frames(instrument=v20setup, plot=True)
frames

We overlay the frame locations onto a spectrum of integrated counts

In [ ]:
plt.ioff()
fig1, ax1 = plt.subplots()
plot(sc.sum(sc.sum(ds["reference"], 'x'), 'y'), ax=ax1)
for i in range(len(frames["GP2"]["left_edges"])):
    ax1.axvspan(frames["GP2"]["left_edges"][i], frames["GP2"]["right_edges"][i], color="C{}".format(i), alpha=0.3)
fig1.canvas.draw_idle()
fig1.canvas

Extract the sections in the original data using value-based slicing and shift the coordinates:

In [ ]:
stitched = wfm.stitch(data=ds, dim="t", frames=frames["GP2"], nbins=256)
stitched

In [ ]:
# plot({"frame{}".format(i): sc.sum(sc.sum(sections["reference"][i], 'x'), 'y')
#       for i in range(len(sections["reference"]))})

To stitch the data, we make a common container with a TOF axis spanning the entire range, and sum the counts from the different frames.

In [ ]:
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Crop to relevant tof section

In [ ]:
tof_start = 9.0e3*sc.units.us
tof_end = 2.6e4*sc.units.us
stitched = stitched["tof", tof_start:tof_end].copy()
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Transmission Masking

Divides the integrated sample counts with an open beam reference. Any values > masking threshold will be masked. The adj pixels step checks for random pixels which were left unmasked or masked with all their neighbours having the same mask value. These are forced to True or false depending on their neighbour value.

In [ ]:
integrated = sc.sum(stitched, 'tof')
integrated /= integrated["reference"]

In [ ]:
masking_threshold = 0.80 * sc.units.one

for key in ["sample", "sample_elastic"]:
    mask = sc.greater(integrated[key].data, masking_threshold)
    # Apply some neighbour smoothing to the masks
    mask = utils.operations.mask_from_adj_pixels(mask)
    stitched[key].masks["non-sample-region"] = mask
stitched

In [ ]:
plot(sc.sum(stitched["sample"], "tof"))

## Normalization

In [ ]:
# Normalize by open beam
normalized = stitched / stitched["reference"]
del normalized["reference"]

replacement = sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan": replacement, "posinf": replacement, "neginf": replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

plot(sc.sum(sc.sum(normalized, 'x'), 'y'))

## Convert to wavelength

In [ ]:
wavelength = sc.neutron.convert(normalized, "tof", "wavelength")

# Rebin to common wavelength axis
edges = sc.array(dims=["wavelength"],
                       values=np.linspace(2.0, 5.0, 119), unit=sc.units.angstrom)
wavelength = sc.rebin(wavelength, "wavelength", edges)
wavelength

In [ ]:
plot(sc.sum(sc.sum(wavelength, 'x'), 'y'), grid=True)

## Apply mean filter

In [ ]:
for key in wavelength:
    wavelength[key].data = utils.operations.mean_from_adj_pixels(wavelength[key].data)

### Show difference between sample and elastic

In [ ]:
plot(wavelength["sample"] - wavelength["sample_elastic"])

## Weighted sum of the counts in the sample

We use the unloaded sample as our reference, so we perform a weighted sum (using the inverse of the variances as weights) of the counts from all the pixels to increase statistics

In [ ]:
variances = sc.variances(wavelength["sample"].data)
# values = wavelength["sample"] / variances
# norm = sc.DataArray(data=sc.reciprocal(variances),
#                     masks=dict(da.masks.items()))
# sc.variances(da.data)
sample_summed = sc.nansum(sc.nansum(wavelength["sample"] / variances, 'x'), 'y') / sc.nansum(sc.nansum(sc.reciprocal(variances), 'x'), 'y')
sample_summed

In [ ]:
plot(sample_summed)

## Group detector pixels to increase signal to noise

In [ ]:
wavelength = sc.rebin(wavelength, "x",
                sc.array(dims=['x'], values=np.linspace(-0.011, 0.011, 33), unit=sc.units.m))
wavelength = sc.rebin(wavelength, "y",
                sc.array(dims=['y'], values=np.linspace(-0.011, 0.011, 33), unit=sc.units.m))
plot(wavelength["sample"])

In [ ]:
plot({"sample": sample_summed, "elastic": wavelength["sample_elastic"]["x", 16]["y", 16]})

# Fitting
## Calculate expected Bragg edge positions 

In [ ]:
# Bragg edge position, in Angstrom, taken from COD entry 9008469
FCC_a = 3.5

# These miller indices for the given unit cell yield bragg edges
# between the maximum and minimum wavelength range.
indices_FCC = [(1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)]

Bragg_edges_FCC = utils.create_Braggedge_list(FCC_a, indices_FCC)

## Bragg-edge fitting

Fit Bragg-edge peaks for the sample initially. Then take this value to find the position within a window
for the elastic sample.